In [2]:
# import
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import requests
import csv
import os
from PIL import Image
from io import BytesIO

In [42]:
# 최종 ver.

# 가톨릭대학교 인천성모병원 url
url = 'https://www.cmcism.or.kr/treatment/treatment_list'

all_doctor_info_list = []
doctor_info_list = []

# 1. chrome driver 띄우기

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--ssl-version-max=tls1.2")
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

for i in range(5, 6):  # <tr> 태그를 7번 반복
    for j in range(1, 6):  # <td> 태그를 6번 반복
        doctor_info_list = []

        tr_elements = driver.find_elements(By.XPATH, '//*[@id="right"]/div[2]/div/table/tbody/tr') 
        tr_element = tr_elements[i]

        td_elements = tr_element.find_elements(By.TAG_NAME, 'td')
        
        if j < len(td_elements):
            department_button = td_elements[j].find_element(By.TAG_NAME, 'a')
            department_button.click()

            doctor_button = driver.find_element(By.LINK_TEXT, '의료진 소개')
            doctor_button.click()

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            team_list_div = soup.find('div', class_='team_list')
            doctor_divs = team_list_div.find_all('div')



            for k in range(0, len(doctor_divs), 2):
                doctor_div = doctor_divs[k]
                
                dptm_elem = soup.find('h5').text if soup.find('h5') else ''
                dptm = dptm_elem  
                
                name_elem = doctor_div.select_one('.p_h a').text if doctor_div.select_one('.p_h a') else ''
                name = name_elem
                
                clinic_elem = doctor_div.select_one('td[style="word-break:keep-all"]').get_text(strip=True) if doctor_div.select_one('td[style="word-break:keep-all"]') else ''
                clinic = clinic_elem
                
                title_elem = doctor_div.select_one('.right_view li:nth-of-type(1)')
                if title_elem:
                    title = title_elem.get_text(strip=True)
                else:
                    title = ''
                
                img_src = doctor_div.find('img')['src'] if doctor_div.find('img') else ''
                img_url = "https://www.cmcism.or.kr" + img_src
                img_name = "가톨릭대학교" + " " + "인천성모병원" + " " + dptm_elem + " " + name_elem + ".png"
                
                # try:
                #     response = requests.get(img_url)
                #     if response.status_code == 200:
                #         image_data = response.content
                #         image = Image.open(BytesIO(image_data))
                #         image.save(f"06_2/{img_name}")
                #     else:
                #         print(f"이미지 다운로드 실패. HTTP 상태 코드: {response.status_code}")
                # except Exception as e:
                #     print(f"이미지 다운로드 중 오류 발생: {str(e)}")
                #     continue  # 예외가 발생하면 다음 반복으로 넘어감
                
                doctor_info = {
                    'dptm': dptm,
                    'name': name,
                    'img_name': img_name, 
                    'clinic': clinic,
                    'title': title
                }
                doctor_info_list.append(doctor_info)
            all_doctor_info_list.extend(doctor_info_list)
    
        driver.back()
        time.sleep(3)
        driver.back()
        time.sleep(3)

driver.quit()

with open("06/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'title', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)

    

In [50]:
title_elem = doctor_div.find('ul', class_="mgb20")
title_elem

<ul class="mgb20">
<!--li>
                                                <span>직급/직위</span>
                                                임상강사                                            </li-->
<li>
<table>
<tbody><tr>
<th>전문분야</th>
<td style="word-break:keep-all">기도질환(만성 폐쇄성 폐질환, 천식), 간질성 폐질환, 감염성 폐질환(폐렴, 결핵, 폐농양), 폐결절</td>
</tr>
</tbody></table>
</li>
</ul>

In [48]:
# "교수, 과장, 뇌과학중개연구소장" 정보 추출
title_elem = doctor_div.select_one('.right_view li span')
if title_elem:
    title = title_elem.text
else:
    title = ''

title

''

In [ ]:
# ver.1

# 가톨릭대학교 인천성모병원 url
url = 'https://www.cmcism.or.kr/treatment/treatment_list'

all_doctor_info_list = []
doctor_info_list = []

# 1. chrome driver 띄우기

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--ssl-version-max=tls1.2")
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

for j in range (0, 6):
    
    doctor_info_list = []


    # element_xpath = f'//*[@id="right"]/div[2]/div/table/tbody/tr[1]/td[{j}]/a'
    # element = driver.find_element(By.XPATH, element_xpath)
    # driver.execute_script("arguments[0].click();", element)

    department_button = driver.find_elements(By.TAG_NAME, 'td')[j]
    department_button.click()

    doctor_button = driver.find_element(By.LINK_TEXT, '의료진 소개')
    # doctor_button = driver.find_element(By.CSS_SELECTOR, '#right > div.sub_wrap > div.Team_nav.Team_nav_type02.mgt30 > ul > li:nth-child(3) > a')

    doctor_button.click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    team_list_div = ""

    team_list_div = soup.find('div', class_='team_list')
    doctor_divs = team_list_div.find_all('div')

    for i in range(0, len(doctor_divs), 2):
        doctor_div = doctor_divs[i]
        
        dptm_elem = soup.find('h5').text if soup.find('h5') else ''
        dptm = dptm_elem  
        
        name_elem = doctor_div.select_one('.p_h a').text if doctor_div.select_one('.p_h a') else ''
        name = name_elem
        
        clinic_elem = doctor_div.select_one('td[style="word-break:keep-all"]').get_text(strip=True) if doctor_div.select_one('td[style="word-break:keep-all"]') else ''
        clinic = clinic_elem
        
        img_src = doctor_div.find('img')['src'] if doctor_div.find('img') else ''
        img_url = "https://www.cmcism.or.kr" + img_src
        img_name = "가톨릭대학교" + " " + "인천성모병원" + " " + dptm_elem + " " + name_elem + ".png"
        
        try:
            response = requests.get(img_url)
            if response.status_code == 200:
                image_data = response.content
                image = Image.open(BytesIO(image_data))
                image.save(f"06_2/{img_name}")
            else:
                print(f"이미지 다운로드 실패. HTTP 상태 코드: {response.status_code}")
        except Exception as e:
            print(f"이미지 다운로드 중 오류 발생: {str(e)}")
            continue  # 예외가 발생하면 다음 반복으로 넘어감
        
        doctor_info = {
            'dptm': dptm,
            'name': name,
            'img_name': img_name, 
            'clinic': clinic,
        }
        doctor_info_list.append(doctor_info)
    all_doctor_info_list.extend(doctor_info_list)
    
    driver.back()
    time.sleep(3)
    driver.back()
    time.sleep(3)
    
    
driver.quit()

with open("06_2/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)

In [5]:
# 개별진료과

url = 'https://www.cmcism.or.kr/treatment/treatment_team?deptSeq=114#'

all_doctor_info_list = []
doctor_info_list = []

# 1. chrome driver 띄우기

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--ssl-version-max=tls1.2")
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

team_list_div = "" 

team_list_div = soup.find('div', class_='team_list')
doctor_divs = team_list_div.find_all('div')

for i in range(0, len(doctor_divs), 2):
    doctor_div = doctor_divs[i]
    
    dptm_elem = soup.find('h5').text if soup.find('h5') else ''
    dptm = dptm_elem  
    
    name_elem = doctor_div.select_one('.p_h a').text if doctor_div.select_one('.p_h a') else ''
    name = name_elem
    
    clinic_elem = doctor_div.select_one('td[style="word-break:keep-all"]').get_text(strip=True) if doctor_div.select_one('td[style="word-break:keep-all"]') else ''
    clinic = clinic_elem
    
    img_src = doctor_div.find('img')['src'] if doctor_div.find('img') else ''
    img_url = "https://www.cmcism.or.kr" + img_src
    img_name = "가톨릭대학교" + " " + "인천성모병원" + " " + dptm_elem + " " + name_elem + ".png"
    
    try:
        response = requests.get(img_url)
        if response.status_code == 200:
            image_data = response.content
            image = Image.open(BytesIO(image_data))
            image.save(f"06/{img_name}")
        else:
            print(f"이미지 다운로드 실패. HTTP 상태 코드: {response.status_code}")
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {str(e)}")
        continue  # 예외가 발생하면 다음 반복으로 넘어감
    
    doctor_info = {
        'dptm': dptm,
        'name': name,
        'img_name': img_name, 
        'clinic': clinic,
    }
    doctor_info_list.append(doctor_info)
all_doctor_info_list.extend(doctor_info_list)

button = driver.find_element(By.LINK_TEXT, '진료과')
button.click()
time.sleep(3)


driver.quit()

with open("06/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)

이미지 다운로드 중 오류 발생: cannot identify image file <_io.BytesIO object at 0x0000023831D372E0>


In [134]:
# 해당 의료진의 "더보기" 버튼을 클릭
more_info_button = driver.find_element(By.)
more_info_button.click()

In [121]:
# "경력" 탭 클릭
career_tab = driver.find_element(By.CSS_SELECTOR, 'li[rel="tab1_2"]')
career_tab.click()

In [130]:
# 새로 열린 팝업 창의 HTML을 파싱
popup_html = driver.page_source
popup_soup = BeautifulSoup(popup_html, 'html.parser')

# 각 정보 추출
position_elem = popup_soup.select_one('table.mgt10 tr td:nth-of-type(2)')
career_elems = popup_soup.select('dl.dl_type02:nth-of-type(1) dd')
edu_elems = popup_soup.select('dl.dl_type02:nth-of-type(2) dd')
academy_elems = popup_soup.select('dl.dl_type02:nth-of-type(3) dd')
prize_elems = popup_soup.select('dl.dl_type02:nth-of-type(4) dd')

# 각 항목별로 정보 가져오기
career_elem = '\n'.join(career_elem.get_text(strip=True) for career_elem in career_elems)
edu_elem = '\n'.join(edu_elem.get_text(strip=True) for edu_elem in edu_elems)
academy_elem = '\n'.join(academy_elem.get_text(strip=True) for academy_elem in academy_elems)
prize_elem = '\n'.join(prize_elem.get_text(strip=True) for prize_elem in prize_elems)


# 정보가 있는 경우에만 데이터를 업데이트
if position_elem:
    position = position_elem.get_text(strip=True)
if career_elem:
    career = career_elem
if edu_elem:
    edu = edu_elem
if academy_elem:
    academy = academy_elem
if prize_elem:
    prize = prize_elem
    
print(position)
print(career)
print(edu)
print(academy)
print(prize)


삼성서울병원 가정의학과 전공의
고대구로병원 가정의학과 임상강사
고대구로병원 가정의학과 임상조교수
가정의학과 전문의
노인의학 분과 인정의(대한노인병학회)
現 가톨릭대학교 인천성모병원 가정의학과 임상교수/과장
강남세브란스병원 인턴
강남세브란스병원 레지던트
원주세브란스병원 촉탁의
강남세브란스병원 임상강사
중앙대병원 임상진료교수
가톨릭대학교 인천성모병원 가정의학과 임상조교수
現 가톨릭대학교 인천성모병원 가정의학과 임상부교수
2014. Relationship between blood mercury concentrations and serum γ-glutamyltranspeptidase level in Korean adults using data from the 2010 Korean National Health and Nutrition Examination Survey. Clinica Chimna Acta
2015. Body weight status misperception and its association with weight control behaviours, depressive mood and psychological distress in nulliparous normal-weight young women. Annals of Human biology
2015. Sex differences in the relationship between blood mercury concentration and metabolic syndrome risk. J Endocrinol Invest
2015. Breakthrough cancer pain. The Korean journal of Hospice and Palliactive Care
2015. High-Dose Vitamin C Promotes Regression of Multiple Pulmonary Metastases Originating from Hepatocellular Carcinoma. Yonsei Med J
201

In [103]:
doctor_pop_divs = soup.find_all('div', class_='doc_layer_pop')

position_elem =doctor_pop_divs[0].find('table', class_='mgt10').find('tr').find_all('td')[1].get_text(strip=True)

IndexError: list index out of range

In [108]:
doctor_pop_divs = soup.find_all('div', class_='doc_layer_pop')

for i in range(len(doctor_pop_divs)):
    doctor_pop_div = doctor_pop_divs[i]
    
    # 직급/직위 정보 추출
    position_elem = doctor_pop_div.select_one('table.mgt10 tr td:nth-of-type(2)').get_text(strip=True)

    # 경력 정보 추출
    career_elem = doctor_pop_div.select_one('dl.dl_type02:nth-of-type(1) dd').get_text(strip=True)
    
    # 학력 정보 추출
    edu_elem = doctor_pop_div.select_one('dl.dl_type02:nth-of-type(2) dd').get_text(strip=True)
    
    # 학회활동 정보 추출
    academy_elem = doctor_pop_div.select_one('dl.dl_type02:nth-of-type(3) dd').get_text(strip=True)
    
    # 수상 정보 추출
    prize_elem = doctor_pop_div.select_one('dl.dl_type02:nth-of-type(4) dd').get_text(strip=True)
    
    try: 
        if position_elem and career_elem and edu_elem and academy_elem and prize_elem:
            position = position_elem                
            career = career_elem
            edu = edu_elem
            academy = academy_elem
            prize = prize_elem

    except:
        position = ""               
        career = ""
        edu = ""
        academy = ""
        prize = ""

    
    print(position)
    print(career)
    print(edu)
    print(academy)
    print(prize)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [77]:
doctor_pop_divs = soup.find_all('div', class_='doc_layer_pop')

for doctor_pop_div in doctor_pop_divs:

    # 직급/직위 정보 추출
    position = doctor_pop_div.find('table', class_='mgt10').find('tr').find_all('td')[1].get_text(strip=True)
    print(position)

    # 경력 정보 추출
    career = doctor_pop_div.find_all('dl', class_='dl_type02')[0].find('dd').get_text(strip=True)
    print(career)

    # 학력 정보 추출
    edu = doctor_pop_div.find_all('dl', class_='dl_type02')[1].find('dd').get_text(strip=True)
    print(edu)

    # 학회활동 정보 추출
    academy = doctor_pop_div.find_all('dl', class_='dl_type02')[2].find('dd').get_text(strip=True)
    print(academy)

    # 수상 정보 추출
    prize = doctor_pop_div.find_all('dl', class_='dl_type02')[3].find('dd').get_text(strip=True)
    print(prize)


IndexError: list index out of range

In [ ]:
# 진료과
# 교수명
# 사진파일명

# 직책

# 전문분야

# 학력
# 경력
# 학회활동
# 기타 (수상)

# 사진 링크

all_doctor_info_list = []
doctor_info_list = []

doc_info = soup.find('div', class_='doc_layer_pop')

for i in range(0, len(doctor_divs), 2):
    doctor_div = doctor_divs[i]
    dptm_elem = soup.find('h5').text
    name_elem = doctor_div.select_one('.p_h a').text
    clinic_elem = doctor_div.select_one('td[style="word-break:keep-all"]').get_text(strip=True)
    img_src = doctor_div.find('img')['src']
    
    try: 
        if dptm_elem and name_elem and img_src and clinic_elem:
            dptm = dptm_elem                
            name = name_elem
            img_url = "https://www.cmcism.or.kr" + img_src
            img_name = "가톨릭대학교" + " " + dptm_elem + " " + name_elem+".png" 
            clinic = clinic_elem

    except:
        dptm = " "
        name = " "
        img_src = " "
        img_name = " "
        clinic = " "
    
    doctor_info = {
        'dptm': dptm,
        'name': name,
        'img_name': img_name, 
        'clinic': clinic,
        'img_url': img_url
    }
    doctor_info_list.append(doctor_info)
all_doctor_info_list.extend(doctor_info_list)

all_doctor_info_list